# 0

In [1]:
!pip install schedule
!pip install pmdarima
!pip install dash
!pip install dash_mantine_components
!pip install dash_iconify

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 46.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 12.0 MB/s eta 0:00:00


# Data Preprocessing

In [2]:
import bw_class
import database_bw
import hi_preprocessing_rfm
import hi_auto_arima
import hi_database
import hi_class

In [3]:
#train_0 = database_bw.making_dataframe_train_db('train_table')
#test_0 = database_bw.making_dataframe_test_db('test_table')

# train_db = "TRAIN.DB"

hi_database.check_tables("TRAIN.DB")
train = hi_database.db_to_df(db_name = "TRAIN.DB", table_name = "train_table")
test = hi_database.db_to_df(db_name = "TEST.DB", table_name = "test_table")
train

,IND,고객ID,거래ID,거래날짜,제품ID,제품카테고리,수량,평균금액,배송료,쿠폰상태,월,성별,고객지역,가입기간,쿠폰코드,할인율,GST,오프라인비용,온라인비용,마케팅비용
0,0,USER_1358,Transaction_0000,2019-01-01,Product_0981,Nest-USA,1,153.71,6.50,Used,1,남,Chicago,12,ELEC10,10.0,0.10,4500,2424.50,6924.50
1,1,USER_1358,Transaction_0001,2019-01-01,Product_0981,Nest-USA,1,153.71,6.50,Used,1,남,Chicago,12,ELEC10,10.0,0.10,4500,2424.50,6924.50
2,2,USER_1358,Transaction_0002,2019-01-01,Product_0904,Office,1,2.05,6.50,Used,1,남,Chicago,12,OFF10,10.0,0.10,4500,2424.50,6924.50
3,3,USER_1358,Transaction_0003,2019-01-01,Product_0203,Apparel,5,17.53,6.50,Not Used,1,남,Chicago,12,SALE10,10.0,0.18,4500,2424.50,6924.50
4,4,USER_1358,Transaction_0003,2019-01-01,Product_0848,Bags,1,16.50,6.50,Used,1,남,Chicago,12,AIO10,10.0,0.18,4500,2424.50,6924.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44456,44456,USER_0035,Transaction_20090,2019-10-31,Product_0989,Nest-USA,2,149.00,6.00,Not Used,10,여,New York,26,ELEC10,10.0,0.10,2000,1251.87,3251.87
44457,44457,USER_0035,Transaction_20091,2019-10-31,Product_0981,Nest-USA,1,149.00,6.00,Used,10,여,New York,26,ELEC10,10.0,0.10,2000,1251.87,3251.87
44458,44458,USER_0035,Transaction_20092,2019-10-31,Product_0990,Nest,1,199.00,19.99,Used,10,여,New York,26,NE10,10.0,0.05,2000,1251.87,3251.87
44459,44459,USER_0035,Transaction_20093,2019-10-31,Product_0992,Nest,1,99.00,6.00,Clicked,10,여,New York,26,NE10,10.0,0.05,2000,1251.87,3251.87


In [4]:
# RFM score 및 고객분류 컬럼 만들기
processor = bw_class.RFMProcessor(train)
rfm_without_outliers, rfm_outliers, rfm_without_outliers_log, X_scaled = processor.process_data()

# 적용
processor = bw_class.RFMProcessor(train)
rfm_without_outliers, rfm_outliers, rfm_without_outliers_log, X_scaled = processor.process_data()
processor.fit_clustering(X_scaled, n_clusters=4)
new_data_predictions = processor.predict(train)

# 최종 데이터에 join
cluster_data = bw_class.mapping_cluster(new_data_predictions)
cluster = cluster_data[['고객ID','Recency','Frequency','Monetary','고객분류']]
train_hi = train.merge(cluster, on = '고객ID', how = 'left')

In [5]:
import numpy as np

np.sort(train_hi['고객분류'].unique())

array(['VIP고객', '관심고객', '우수고객', '이탈고객', '잠재고객'], dtype=object)

# Autoarima

In [6]:
hi_pipeline = hi_class.AutoArimaPipeline()
df_arima = hi_pipeline.hi_preprocessing(train_hi)

In [7]:
df_arima

,거래날짜,마케팅비용,Cluster
0,2019-01-01,6924.50,VIP고객
1,2019-01-01,6924.50,VIP고객
2,2019-01-01,6924.50,VIP고객
3,2019-01-01,6924.50,VIP고객
4,2019-01-01,6924.50,VIP고객
...,...,...,...
44456,2019-10-31,3251.87,우수고객
44457,2019-10-31,3251.87,우수고객
44458,2019-10-31,3251.87,우수고객
44459,2019-10-31,3251.87,우수고객


In [8]:
# data split
out = hi_pipeline.split_data_by_cluster(df_arima)
train0, test0 = hi_pipeline.create_train_test_by_cluster(train_hi, out)

In [9]:
# 모델 피팅
hi_pipeline.fit(train0)

# 예측 수행
predictions = hi_pipeline.predict(test0)

# 결과 출력
for i, prediction in enumerate(predictions):
    print(f"Predictions for Cluster {i}:")
    print(prediction)

Predictions for Cluster 0:
              마케팅비용 Cluster         pred
거래날짜                                    
2019-08-28  5178.57   VIP고객  4987.677714
2019-08-31  4296.81   VIP고객  4476.458705
2019-09-01  5424.97   VIP고객  5080.732193
2019-09-02  4307.62   VIP고객  4503.957045
2019-09-03  4506.42   VIP고객  4524.379496
2019-09-04  3755.01   VIP고객  4069.861854
2019-09-05  3701.09   VIP고객  3957.461096
2019-09-06  3792.58   VIP고객  3993.312783
2019-09-08  3862.10   VIP고객  4041.305558
2019-09-11  4683.99   VIP고객  4549.813083
2019-09-12  4117.36   VIP고객  4292.411808
2019-09-13  4079.34   VIP고객  4224.407581
2019-09-14  4408.06   VIP고객  4412.395070
2019-09-15  5546.67   VIP고객  5138.196235
2019-09-16  4514.85   VIP고객  4637.230654
2019-09-17  5287.02   VIP고객  5017.558812
2019-09-18  4797.87   VIP고객  4789.476901
2019-09-19  3572.89   VIP고객  4006.796660
2019-09-21  3598.20   VIP고객  3882.221785
2019-09-22  4395.73   VIP고객  4343.911697
2019-09-23  5408.10   VIP고객  5039.353481
2019-09-24  4310.85   VIP고객  4

# Visualization

In [11]:
import plotly.graph_objs as go
import pandas as pd

class hi_visualizer:
    def __init__(self, data, clusters, marketing_cost_column='마케팅비용', prediction_column='pred'):
        """
        Initializes the visualizer with data and clusters.

        Parameters:
        - data: list of pd.DataFrame, list of datasets containing actual marketing costs (last element is the overall dataset)
        - clusters: list of pd.DataFrame, list of datasets containing predictions (last element is the overall dataset)
        - marketing_cost_column: str, the column name for the actual marketing costs in the data dataset
        - prediction_column: str, the column name for the predicted marketing costs in the clusters dataset
        """
        self.data = data
        self.clusters = clusters
        self.marketing_cost_column = marketing_cost_column
        self.prediction_column = prediction_column

        # Convert index to datetime if not already
        for df in self.data:
            df.index = pd.to_datetime(df.index)
        for df in self.clusters:
            df.index = pd.to_datetime(df.index)

    def visualize_overall(self):
        """
        Visualizes marketing costs and predictions over time for the entire dataset.

        Returns:
        - A plotly.graph_objs.Figure object for the overall data
        """
        # Initialize the plotly figure
        fig = go.Figure()

        # Add trace for overall actual marketing costs
        overall_data = self.data[-1]
        overall_clusters = self.clusters[-1]

        fig.add_trace(go.Scatter(
            x=overall_data.index,
            y=overall_data[self.marketing_cost_column],
            mode='lines+markers',
            name='Actual Marketing Cost (Overall)'
        ))

        # Add trace for overall predicted marketing costs
        fig.add_trace(go.Scatter(
            x=overall_clusters.index,
            y=overall_clusters[self.prediction_column],
            mode='lines+markers',
            name='Predicted Marketing Cost (Overall)'
        ))

        # Update layout
        fig.update_layout(
            title='Marketing Costs and Predictions Over Time (Overall)',
            xaxis_title='Transaction Date',
            yaxis_title='Marketing Cost',
            legend_title='Legend',
            hovermode='x unified'
        )

        return fig

    def visualize_cluster(self, cluster_number):
        """
        Visualizes marketing costs and predictions over time for a specific cluster.

        Parameters:
        - cluster_number: int, the cluster number to visualize

        Returns:
        - A plotly.graph_objs.Figure object for the specified cluster
        """
        if cluster_number >= len(self.data) - 1:
            raise ValueError("Invalid cluster number. It should be less than the total number of clusters.")

        # Initialize the plotly figure
        fig = go.Figure()

        # Add traces for the specified cluster
        cluster_data = self.data[cluster_number]
        cluster_predictions = self.clusters[cluster_number]

        # Actual marketing cost for the cluster
        fig.add_trace(go.Scatter(
            x=cluster_data.index,
            y=cluster_data[self.marketing_cost_column],
            mode='lines+markers',
            name=f'Actual Marketing Cost - Cluster {cluster_number}'
        ))

        # Predicted marketing cost for the cluster
        fig.add_trace(go.Scatter(
            x=cluster_predictions.index,
            y=cluster_predictions[self.prediction_column],
            mode='lines+markers',
            name=f'Predicted Marketing Cost - Cluster {cluster_number}'
        ))

        # Update layout
        fig.update_layout(
            title=f'Marketing Costs and Predictions Over Time (Cluster {cluster_number})',
            xaxis_title='Transaction Date',
            yaxis_title='Marketing Cost',
            legend_title='Legend',
            hovermode='x unified'
        )

        return fig

# Example usage:
# visualizer = hi_visualizer(train, predictions)

# Visualize overall data
# overall_fig = visualizer.visualize_overall()
# overall_fig.show()

# Visualize cluster 0
# cluster_fig = visualizer.visualize_cluster(0)
# cluster_fig.show()


In [13]:
visualizer = hi_visualizer(train0, predictions)
overall_fig = visualizer.visualize_overall()
overall_fig.show()
cluster_fig = visualizer.visualize_cluster(4)  # Visualize cluster 5
cluster_fig.show()
